## Assignment 1
Procedure 1: 1) Fundamental data structures. 2) Tokenize / BOW. 3) Separate NB for each emotion class and Evaluation.

Procedure 2 1) Test pairwise emotion labels (e1, e2) if prediction can be improved by adding a probability (pe1|e2). 2) Search through all possible pairwise label combinations to find highest probability + evaluation.

Procedure 3 (MSC) 1) Evaluate performance for prediction of all emotion labels jointly. 2) Inference: Gibbs Sampling (or equivalent)

## Procedure 1 - Corpus Cleaning + Tokenization

In [16]:
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import pandas as pd
import numpy as np
import csv
import os
import string
#
class Document:
    """
    This class oversees reading, cleaning and formatting of the text corpus
    """
    #
    def __init__(self, path=None):
        if path is not None:
            BASE_DIR = os.path.join( os.path.dirname(os.getcwd()))
            self.path = BASE_DIR + path
        # Emotion Categories (Classes)
        self.emotions = ("anger","anticipation","disgust","fear","joy","sadness","surprise","trust")
        self.stemmer = PorterStemmer()
        self.lemmatizer = WordNetLemmatizer()
    #
    def __open_file_and_return_dump(self):
        """ 
        Opens file and returns a dump containing all the data.
        The data structure used to record the dumped data is a list of strings,
        where each string represents a single record).
        """
        corpus = []
        with open(self.path) as csvfile:
            rows = csv.reader(csvfile)
            for row in rows:
                temp_string = ""
                for item in row:
                    temp_string += str(item) + " "
                corpus.append(temp_string)
        return corpus
    #
    def __split_labels_from_vectors(self):
        """
        Takes corpus as input, and splits the data into respective labels and training data.
        Returns this data as two separate lists
        """
        #
        corpus = self.__open_file_and_return_dump()
        label_count = len(self.emotions)
        character_splitter = '\t'
        train_y, train_X = [], []
        for row in corpus:
            groups = row.split(character_splitter)
            train_y.append(character_splitter.join(groups[:label_count]))
            train_X.append(character_splitter.join(groups[label_count:]))
        return train_y, train_X
    #
    def __tokenize_corpus(self, data, delimeter=" "):
        """
        Takes a list as input, and tokenizes the data.
        """
        corpus = []
        [(corpus.append(row.split(delimeter))) for row in data]
        return corpus
    #
    def __replace(self, word, symbols, placeholder=""):
        """
        An overriding of the original python method, so as to replace all characters 
        in a string based on whether they occur in a list.
        """
        temp_string = word
        for symbol in symbols:
            temp_string = temp_string.replace(symbol, placeholder)
        return temp_string
    #
    def __remove_stop_words(self, data):
        """
        Takes a list of data, and iterates over each element to 
        scan and remove stop words.

        The method ensures to convert all text instances to lowercase.

        The method ensures to remove jargon symbols (#,%,&,etc).

        This method ensures to remove punctuation.
        """
        temp_list, jargon_symbols = [], ('%','$','#','@','^','&','*','(',')','+','/','\'','!','?','.',',',':',';','~','0','1','2','3','4','5','6','7','8','9')
        for row in data:
            filtered_words = [self.__replace(word.lower().translate(string.punctuation), symbols=jargon_symbols) for word in row if word.lower() not in stopwords.words('english')]          
            temp_list.append(filtered_words)
        #
        new_list = []
        for row in temp_list:
            row_list = []
            for word in row:
                if word != "":
                    row_list.append(word)
            new_list.append(row_list)
        #
        return new_list
    #
    def __word_morhpology(self, data, stemming=0, lemmatizing=0):
        temp_list = []
        for row in data:
            filtered_words = row
            if stemming == 1:
                # Stemming
                filtered_words = [self.stemmer.stem(word) for word in filtered_words]
            #
            if stemming == 1:
                # Lemmatizing
                filtered_words = [self.lemmatizer.lemmatize(word) for word in filtered_words]
            #
            temp_list.append(filtered_words)
        #
        return temp_list
    #
    def get_emotions(self):
        return self.emotions
    #
    def clean_corpus(self):
        train_y, train_X = self.__split_labels_from_vectors()
        tokenized_X, tokenized_y = self.__tokenize_corpus(data=train_X), self.__tokenize_corpus(data=train_y, delimeter="\t")
        cleaned_tokenized_X, cleaned_tokenized_y = self.__remove_stop_words(data=tokenized_X), self.__remove_stop_words(data=tokenized_y)
        cleaned_tokenized_X = self.__word_morhpology(cleaned_tokenized_X,1,1)
        return cleaned_tokenized_X, cleaned_tokenized_y
    #
    def binarize(self, y):
        value = "---"
        y_binarized = []
        for i in range(len(y)):
            y_binarized.append([])
            for element in y[i]:
                if value in element:
                    y_binarized[i].append(0)
                else:
                    y_binarized[i].append(1)
        return y_binarized
#
training_document = Document(path="\\data\\ssec-aggregated\\train-combined-0.0.csv")
cleaned_tokenized_train_X, cleaned_tokenized_train_y = training_document.clean_corpus()
#
# This will be used later for evaluation purposes
testing_document = Document(path= "\\data\\ssec-aggregated\\test-combined-0.0.csv")
cleaned_tokenized_test_X, cleaned_tokenized_test_y = testing_document.clean_corpus()
binarized_cleaned_tokenized_test_y = testing_document.binarize(cleaned_tokenized_test_y)
#
print("First 5 lines of training corpus: \n" + str(cleaned_tokenized_train_X[0:5]) + "\n")
print("First 5 lines of training labels: \n" + str(cleaned_tokenized_train_y[0:5]) + "\n")
print("First 5 lines of testing labels: \n" + str(cleaned_tokenized_test_y[0:5]) + "\n")
print("First 5 lines of testing binarized labels: \n" + str(binarized_cleaned_tokenized_test_y[0:5]) + "\n")
print("Shape of training corpus: \n" + str(np.shape(cleaned_tokenized_train_X)))
print("Shape of training corpus: \n" + str(np.shape(cleaned_tokenized_train_y)))
print("Shape of testing corpus: \n" + str(np.shape(cleaned_tokenized_test_y)))
print("Shape of testing corpus: \n" + str(np.shape(binarized_cleaned_tokenized_test_y)))

First 5 lines of training corpus: 
[['tedcruz', 'handovertheserv', 'wipe', 'clean', 'k', 'delet', 'email', 'explain', 'derelict', 'dutyli', 'benghazi', 'etc', 'tcot'], ['hillari', 'best', 'choic', 'truli', 'want', 'continu', 'progress', 'nation', 'ohio'], ['theview', 'think', 'countri', 'readi', 'femal', 'pre', 'cant', 'ever', 'hillari'], ['gave', 'unhealthi', 'amount', 'hard-earn', 'money', 'away', 'big', 'govt', 'untrustworthi', 'ir', 'whyimnotvotingforhillari'], ['portiaaboulg', 'thank', 'ad', 'list']]

First 5 lines of training labels: 
[['anger', 'anticipation', 'disgust', 'fear', '---', 'sadness', 'surprise', '---'], ['---', 'anticipation', '---', 'fear', 'joy', '---', '---', 'trust'], ['anger', 'anticipation', 'disgust', '---', 'joy', 'sadness', '---', '---'], ['anger', 'anticipation', 'disgust', 'fear', '---', 'sadness', '---', '---'], ['---', '---', '---', '---', 'joy', '---', '---', '---']]

First 5 lines of testing labels: 
[['anger', 'anticipation', 'disgust', '---', 'joy',

## Procedure 1 - Naive Bayes Class Template

In [17]:
class NaiveBayes:
    """
    This class oversees the Naive Bayes Model Template
    """
    #
    def __init__(self, emotion_index):
        """
        Constructor
        """
        d = Document()
        self.emotions = d.get_emotions()
        self.emotion_index = emotion_index
        if self.emotion_index > len(self.emotions):
            print("Index exceeds the current emotion list capacity. Index musn't exceed " + str(len(self.emotions)))
        self.BOW_0, self.BOW_1 = {}, {} # Separate BOW models (BOW_1 - For likely vocab, BOW_0 - For not likely vocab)
        self.word_count_category_0, self.word_count_category_1 = 0,0 
    #
    def __get_sentiment(self):
        """
        Returns emotion of this instance classifier
        """
        return self.emotions[self.emotion_index]
    #
    def __prod(self,iterable):
        """
        Product multiplier - Accepts list and returns product of all list elements
        """
        product = 1
        for number in iterable:
            product *= number
        return product
    #
    def __calculate_NB(self, liklihood_prob, prior_prob):
        return self.__prod(liklihood_prob) * prior_prob
    #
    def __calculate_pairwise_NB(self, e1_liklihood_prob, e2_liklihood_prob, e1_prior_prob, e2_prior_prob, pairwise_prob):
        return pairwise_prob * e1_prior_prob * e2_prior_prob * self.__prod(e1_liklihood_prob) * self.__prod(e2_liklihood_prob)
    #
    def fit(self, X, y):
        """
        Accepts training data and training labels, which will form the basis of the Naive Bayes model.
        This function oversees cosntruction of the bag of words model. 
        """
        selected_emotion = self.__get_sentiment()
        #
        for i in range(len(y)):
            if selected_emotion in y[i]:
                for word in X[i]:
                    self.word_count_category_1 += 1
                    if word in self.BOW_1:
                        self.BOW_1[word] += 1
                    else:
                        self.BOW_1[word] = 1
            else:
                for word in X[i]:
                    self.word_count_category_0 += 1
                    if word in self.BOW_0:
                        self.BOW_0[word] += 1
                    else:
                        self.BOW_0[word] = 1
    #
    def predict_proba(self, X, pairwise_emotion=None):
        """
        Accepts input document, and returns estimated liklihoods for an emotion holding and not holding.
        
        The method also accepts a second parameter, denoting a pairwise emotion. If the parameter is not empty,
        the liklihood emotions of holding/not holding will be calculated based on P(e1|e2).
        
        X                :text document (sentence to be classified)
        pairwise_emotion :Emotion 2 pairwise model
        
        """
        prob_0, prob_1 = 0,0
        #
        # Calculating Prior Probabilities for initial emotion
        e1_prior_prob = sum(self.BOW_1.values()) / (sum(self.BOW_0.values()) + sum(self.BOW_1.values())) 
        e1_prior_prob_na = sum(self.BOW_0.values()) / (sum(self.BOW_0.values()) + sum(self.BOW_1.values()))
        if pairwise_emotion is not None:
            #
            # Calculating Prior Probabilities for pairwise emotion
            e2_BOW_0, e2_BOW_1 = pairwise_emotion.get_model_BOWs()
            e2_prior_prob = sum(e2_BOW_1.values()) / (sum(e2_BOW_0.values()) + sum(e2_BOW_1.values())) 
            e2_prior_prob_na = sum(e2_BOW_0.values()) / (sum(e2_BOW_0.values()) + sum(e2_BOW_1.values()))
            #
            # Calculating probability of e1 given e2
            e1_count, e2_count = 0,0
            for emotions in cleaned_tokenized_train_y:
                if pairwise_emotion.__get_sentiment() in emotions:
                    e2_count += 1
                    if self.__get_sentiment() in emotions:
                        e1_count += 1
            pairwise_prob = e1_count / e2_count
        #
        # Calculating the liklihood probability of having this emotion
        e1_likelihood_probabilities, e2_likelihood_probabilities = [], []
        for word in X:
            if word in self.BOW_1:
                e1_likelihood_probability = self.BOW_1[word] / e1_prior_prob
                e1_likelihood_probabilities.append(e1_likelihood_probability)
            if pairwise_emotion is not None and word in e2_BOW_1:
                e2_likelihood_probability = e2_BOW_1[word] / e2_prior_prob
                e2_likelihood_probabilities.append(e2_likelihood_probability)
        #
        if pairwise_emotion is not None:
            prob_1 = self.__calculate_pairwise_NB(e1_liklihood_prob=e1_likelihood_probabilities, 
                                                  e2_liklihood_prob=e2_likelihood_probabilities, 
                                                  e1_prior_prob=e1_prior_prob, 
                                                  e2_prior_prob=e2_prior_prob,
                                                  pairwise_prob=pairwise_prob)
        else:
            prob_1 = self.__calculate_NB(e1_likelihood_probabilities, e1_prior_prob)
        #
        # Calculating the liklihood probability of not having this emotion
        e1_likelihood_probabilities, e2_likelihood_probabilities = [], []
        for word in X:
            if word in self.BOW_0:
                e1_likelihood_probability = self.BOW_0[word] / e1_prior_prob_na
                e1_likelihood_probabilities.append(e1_likelihood_probability)
            if pairwise_emotion is not None and word in e2_BOW_0:
                e2_likelihood_probability = e2_BOW_0[word] / e2_prior_prob_na
                e2_likelihood_probabilities.append(e2_likelihood_probability)
        #
        if pairwise_emotion is not None:
            prob_0 = self.__calculate_pairwise_NB(e1_liklihood_prob=e1_likelihood_probabilities, 
                                                  e2_liklihood_prob=e2_likelihood_probabilities, 
                                                  e1_prior_prob=e1_prior_prob, 
                                                  e2_prior_prob=e2_prior_prob,
                                                  pairwise_prob=pairwise_prob)
        else:     
            prob_0 = self.__calculate_NB(e1_likelihood_probabilities, e1_prior_prob_na)
            
        #
        return prob_0, prob_1
    #
    def predict(self, X, pairwise_emotion=None):
        """
        A wrapper function for: 
        
        self.predict_proba
        
        Returns whether emotion holds or not
        """
        prob_0, prob_1 = self.predict_proba(X,pairwise_emotion)
        if prob_0 > prob_1:
            return 0 # Emotion is not likely
        else:
            return 1 # Emotion is likely
    #
    def get_model_BOWs(self):
        return self.BOW_0, self.BOW_1
        
#
NB_classifier_anger = NaiveBayes(0)
NB_classifier_anger.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_anticipation = NaiveBayes(1)
NB_classifier_anticipation.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_disgust = NaiveBayes(2)
NB_classifier_disgust.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_fear = NaiveBayes(3)
NB_classifier_fear.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_joy = NaiveBayes(4)
NB_classifier_joy.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_sadness = NaiveBayes(5)
NB_classifier_sadness.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_surprise = NaiveBayes(6)
NB_classifier_surprise.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
NB_classifier_trust = NaiveBayes(7)
NB_classifier_trust.fit(X=cleaned_tokenized_train_X, y=cleaned_tokenized_train_y)
#
classifiers = [NB_classifier_anger, 
               NB_classifier_anticipation, 
               NB_classifier_disgust, 
               NB_classifier_fear, 
               NB_classifier_joy, 
               NB_classifier_sadness, 
               NB_classifier_surprise, 
               NB_classifier_trust]
#
# Quick Testing
test = "If you want to live as a healer".split()
pred = NB_classifier_anger.predict(test)
print("Anger: " + str(pred))
pred = NB_classifier_anticipation.predict(test)
print("Anticipation: " + str(pred))
pred = NB_classifier_disgust.predict(test)
print("Disgust: " + str(pred))
pred = NB_classifier_fear.predict(test)
print("Fear: " + str(pred))
pred = NB_classifier_joy.predict(test)
print("Joy: " + str(pred))
pred = NB_classifier_sadness.predict(test)
print("Sadness: " + str(pred))
pred = NB_classifier_surprise.predict(test)
print("Surprise: " + str(pred))
pred = NB_classifier_trust.predict(test)
print("Trust: " + str(pred))

Anger: 1
Anticipation: 1
Disgust: 1
Fear: 0
Joy: 1
Sadness: 0
Surprise: 0
Trust: 0


## Procedure 1 - Individual emotion model evaluation

In [18]:
def true_label_alloc(binarized_cleaned_tokenized_test_y, index):
    """
    Iterates through all correct labels and retrieves only that
    subset which corresponds to the desired emotion.
    """
    y_true = []
    for row in binarized_cleaned_tokenized_test_y:
        y_true.append(row[index])
    return y_true
#
# Anger Model Evaluation
y_true, y_pred = true_label_alloc(binarized_cleaned_tokenized_test_y,0), []
for sentence in cleaned_tokenized_test_X:
    predictions = NB_classifier_anger.predict(sentence)
    y_pred.append(predictions)
print("Anger Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Anger Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Anger Recall: " + str(recall_score(y_true, y_pred) * 100))
print("Anger F-Score: " + str(f1_score(y_true, y_pred) * 100))
print("------------------------------------------------")
#
# Anticipation Model Evaluation
y_true, y_pred = true_label_alloc(binarized_cleaned_tokenized_test_y,1), []
for sentence in cleaned_tokenized_test_X:
    predictions = NB_classifier_anticipation.predict(sentence)
    y_pred.append(predictions)
print("Anticipation Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Anticipation Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Anticipation Recall: " + str(recall_score(y_true, y_pred) * 100))
print("Anticipation F-Score: " + str(f1_score(y_true, y_pred) * 100))
print("------------------------------------------------")
#
# Disgust Model Evaluation
y_true, y_pred = true_label_alloc(binarized_cleaned_tokenized_test_y,2), []
for sentence in cleaned_tokenized_test_X:
    predictions = NB_classifier_disgust.predict(sentence)
    y_pred.append(predictions)
print("Disgust Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Disgust Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Disgust Recall: " + str(recall_score(y_true, y_pred) * 100))
print("Disgust F-Score: " + str(f1_score(y_true, y_pred) * 100))
print("------------------------------------------------")
#
# Fear Model Evaluation
y_true, y_pred = true_label_alloc(binarized_cleaned_tokenized_test_y,3), []
for sentence in cleaned_tokenized_test_X:
    predictions = NB_classifier_fear.predict(sentence)
    y_pred.append(predictions)
print("Fear Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Fear Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Fear Recall: " + str(recall_score(y_true, y_pred) * 100))
print("Fear F-Score: " + str(f1_score(y_true, y_pred) * 100))
print("------------------------------------------------")
#
# Joy Model Evaluation
y_true, y_pred = true_label_alloc(binarized_cleaned_tokenized_test_y,4), []
for sentence in cleaned_tokenized_test_X:
    predictions = NB_classifier_joy.predict(sentence)
    y_pred.append(predictions)
print("Joy Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Joy Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Joy Recall: " + str(recall_score(y_true, y_pred) * 100))
print("Joy F-Score: " + str(f1_score(y_true, y_pred) * 100))
print("------------------------------------------------")
#
# Sadness Model Evaluation
y_true, y_pred = true_label_alloc(binarized_cleaned_tokenized_test_y,5), []
for sentence in cleaned_tokenized_test_X:
    predictions = NB_classifier_sadness.predict(sentence)
    y_pred.append(predictions)
print("Sadness Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Sadness Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Sadness Recall: " + str(recall_score(y_true, y_pred) * 100))
print("Sadness F-Score: " + str(f1_score(y_true, y_pred) * 100))
print("------------------------------------------------")
#
# Surprise Model Evaluation
y_true, y_pred = true_label_alloc(binarized_cleaned_tokenized_test_y,6), []
for sentence in cleaned_tokenized_test_X:
    predictions = NB_classifier_surprise.predict(sentence)
    y_pred.append(predictions)
print("Surprise Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Surprise Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Surprise Recall: " + str(recall_score(y_true, y_pred) * 100))
print("Surprise F-Score: " + str(f1_score(y_true, y_pred) * 100))
print("------------------------------------------------")
#
# Trust Model Evaluation
y_true, y_pred = true_label_alloc(binarized_cleaned_tokenized_test_y,7), []
for sentence in cleaned_tokenized_test_X:
    predictions = NB_classifier_trust.predict(sentence)
    y_pred.append(predictions)
print("Trust Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Trust Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Trust Recall: " + str(recall_score(y_true, y_pred) * 100))
print("Trust F-Score: " + str(f1_score(y_true, y_pred) * 100))
print("------------------------------------------------")

Anger Accuracy: 70.3987730061
Anger Precision: 75.1510574018
Anger Recall: 79.9196787149
Anger F-Score: 77.4620474893
------------------------------------------------
Anticipation Accuracy: 59.6625766871
Anticipation Precision: 68.0555555556
Anticipation Recall: 65.0622406639
Anticipation F-Score: 66.5252439542
------------------------------------------------
Disgust Accuracy: 64.8261758691
Disgust Precision: 61.0671936759
Disgust Recall: 67.7631578947
Disgust F-Score: 64.2411642412
------------------------------------------------
Fear Accuracy: 62.5766871166
Fear Precision: 54.4854881266
Fear Recall: 51.625
Fear F-Score: 53.0166880616
------------------------------------------------
Joy Accuracy: 64.3149284254
Joy Precision: 53.5499398315
Joy Recall: 58.784676354
Joy F-Score: 56.0453400504
------------------------------------------------
Sadness Accuracy: 60.6850715746
Sadness Precision: 62.3728813559
Sadness Recall: 69.3685202639
Sadness F-Score: 65.6849620705
-----------------------

## Procedure 1 - Collective emotion model evaluation

In [19]:
y_true, y_pred = binarized_cleaned_tokenized_test_y, []
for sentence in cleaned_tokenized_test_X:
    predictions = [NB_classifier_anger.predict(sentence),
                   NB_classifier_anticipation.predict(sentence),
                   NB_classifier_disgust.predict(sentence),
                   NB_classifier_fear.predict(sentence),
                   NB_classifier_joy.predict(sentence),
                   NB_classifier_sadness.predict(sentence),
                   NB_classifier_surprise.predict(sentence),
                   NB_classifier_trust.predict(sentence)]
    y_pred.append(predictions)
#
def flatten(iterable):
    flattened_list = []
    for row in iterable:
        for element in row:
            flattened_list.append(element)
    return flattened_list
y_true = flatten(y_true)
y_pred = flatten(y_pred)
#
print("Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Recall: " + str(recall_score(y_true, y_pred) * 100))
print("F-Score: " + str(f1_score(y_true, y_pred) * 100))

Accuracy: 64.5258179959
Precision: 61.171011328
Recall: 62.3539232053
F-Score: 61.7568033069


## Procedure 2 - Pairwise emotion evaluation (Disgust|Anger)

In [20]:
#
# Disgust|Anger Model Evaluation
y_true, y_pred = true_label_alloc(binarized_cleaned_tokenized_test_y,2), []
for sentence in cleaned_tokenized_test_X:
    predictions = NB_classifier_disgust.predict(sentence,NB_classifier_anger)
    y_pred.append(predictions)
print("Disgust|Anger Accuracy: " + str(accuracy_score(y_true, y_pred) * 100))
print("Disgust|Anger Precision: " + str(precision_score(y_true, y_pred) * 100))
print("Disgust|Anger Recall: " + str(recall_score(y_true, y_pred) * 100))
print("Disgust|Anger F-Score: " + str(f1_score(y_true, y_pred) * 100))
print("------------------------------------------------")

Disgust|Anger Accuracy: 65.3374233129
Disgust|Anger Precision: 59.8319327731
Disgust|Anger Recall: 78.0701754386
Disgust|Anger F-Score: 67.7450047574
------------------------------------------------
